In [1]:
options(warn = -1)

In [22]:
library(readxl)
library(lubridate)
library(rgdal)

Loading required package: sp

rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
Overwritten PROJ_LIB was C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj



In [3]:
inpath <- 'T:/Data/COUNTS/Nonmotorized Counts/Summary Tables/Bicycle/'

In [4]:
data <- read.csv(paste0(inpath, 'Bicycle_HourlyForTableau.csv'))

In [17]:
names(data)

[1] "Direction"       "ObsHours"        "Location"        "LocationId"     
 [5] "Day"             "Month"           "MonthDesc"       "Year"           
 [9] "Date"            "Season"          "Weekday"         "IsHoliday"      
[13] "Period"          "DailyCounts"     "IsOneway"        "OnewayDirection"
[17] "IsSidewalk"      "UoInSession"     "IsSpecialEvent"  "UniqueId"       
[21] "Hour"            "Hourly_Count"

In [5]:
data$Date <- as.Date(data$Date, "%Y-%m-%d")

In [6]:
locdata <- read.csv("T:/Data/COUNTS/Nonmotorized Counts/Supporting Data/Supporting Bicycle Data/CountLocationInformation.csv")

In [7]:
# remove missing data
data1 <- data[!is.na(data$Hourly_Count),]

In [8]:
# use only the total direction
data1 <- data1[data1$Direction == 'Total',]

In [9]:
# if the most recent year is not complete, remove it first
data1 <- data1[data1$Year != 2022,]

In [11]:
locvars <- c('Location', 'Latitude', 'Longitude', 'Site_Name', 
             'DoubleCountLocation', 'IsOneway', 'OnewayDirection', 
             'IsSidewalk')

In [23]:
MPOBound <- readOGR(dsn = "V:/Data/Transportation", layer="MPO_Bound")

OGR data source with driver: ESRI Shapefile 
Source: "V:\Data\Transportation", layer: "MPO_Bound"
with 1 features
It has 3 fields


In [24]:
# require MPOBound
df2spdf <- function(df, lon_col_name, lat_col_name, trans = TRUE){
  lonlat <- sp::CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
  lon_col_no <- which(names(df)==lon_col_name)
  lat_col_no <- which(names(df)==lat_col_name)
  xy <- data.frame(df[,c(lon_col_no,lat_col_no)])
  coordinates(xy) <- c(lon_col_name, lat_col_name)
  proj4string(xy) <- lonlat
  spdf <- sp::SpatialPointsDataFrame(coords = xy, data = df)
  if(trans){
    spdf <- spTransform(spdf, CRS(proj4string(MPOBound)))
  }
  return(spdf)
}

In [28]:
path <- "T:/DCProjects/StoryMap/BikeCounting/BikeCounts/Output"

In [32]:
agg_data <- function(var="Hour"){
    outdata <- aggregate(x=list(BPH = data1$Hourly_Count), by=list(Category = data1[,var], Location = data1$Location), FUN=mean)
    outdata <- merge(outdata, locdata[,locvars], by = 'Location')
    names(outdata)[which(names(outdata)=='Category')] <- var
    for(loc in unique(outdata$Location)){
        c <- outdata[outdata$Location == loc, var]
        outdata[outdata$Location==loc,"N"] <- length(c)
    }
    write.csv(outdata, paste0(path, "/BPH_", var,".csv"), row.names = FALSE)
    print(paste("Got the aggregated data by", var))
    outspdf <- df2spdf(outdata, 'Longitude', 'Latitude')
    writeOGR(outspdf, dsn=path, layer=paste0("BPH_", var), 
         driver="ESRI Shapefile", overwrite_layer=TRUE)
    print(paste("Got the spatial aggregated data by", var))
}

In [33]:
agg_data()

[1] "Got the aggregated data by Hour"
[1] "Got the spatial aggregated data by Hour"


In [34]:
for(var in c("Weekday", "Month", "Season")){
    agg_data(var=var)
}

[1] "Got the aggregated data by Weekday"
[1] "Got the spatial aggregated data by Weekday"
[1] "Got the aggregated data by Month"
[1] "Got the spatial aggregated data by Month"
[1] "Got the aggregated data by Season"
[1] "Got the spatial aggregated data by Season"
